results in seem to be wrong ver4_machine_learning_utah_classification_201906122.ipynb

In [1]:
import sys
import os
import subprocess
import re
import numpy as np
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as psql
import getpass
import matplotlib as mpl
import argparse
import glob
import traceback
import hashlib
import math
import collections
import functools

from statsmodels.stats.proportion import proportion_confint as statmodels_proportion_confint

# import sklearn.preprocessing
# import sklearn.feature_selection
# import sklearn.ensemble 
# import sklearn.neural_network
# import sklearn.model_selection
# import sklearn.metrics
# import sklearn.pipeline
from sklearn.externals import joblib

mpl.rcParams['figure.dpi'] = 80

import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display
from slugify import slugify

In [2]:
app_base_dir = '/home/spbproc/euso-spb-patt-reco-v1'
if app_base_dir not in sys.path:
    sys.path.append(app_base_dir)

import event_processing_v3
import event_processing_v4
import postgresql_v3_event_storage
import dataset_query_functions_v3

import tool.acqconv
from data_analysis_utils import *
from data_analysis_utils_dataframes import *
from data_analysis_utils_performance import *
# import supervised_classification as supc    

Welcome to JupyROOT 6.13/02


In [3]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [4]:
# inverse_means_map = np.load('/home/spbproc/euso-spb-patt-reco-v1/resources/inverse_flat_average_directions_4m_flipud.npy')

In [5]:
classification_id = '201906122'
model_data_snippets_dir = 'ver4_machine_learning_w_labeled_flight_' + classification_id
utah_file_analysis_snippets_dir = 'utah_events_directory_analysis'
data_snippets_dir = 'ver4_machine_learning_utah_classification_' + classification_id

os.makedirs(data_snippets_dir, exist_ok=True)

## Selecting the data

In [6]:
event_processing_cls = event_processing_v4.EventProcessingV4
event_v3_storage_provider_utah = dataset_query_functions_v3.build_event_v3_storage_provider(
    event_storage_provider_config_file=os.path.join(app_base_dir,'config.ini'), 
    table_names_version='ver4',
    event_storage_class=postgresql_v3_event_storage.PostgreSqlEventV3StorageProvider,
    event_processing_class=event_processing_cls,
    readonly=False,
    schema_name_overwrite = 'miso2_2_spb_processing_v4'
)

query_functions_utah = dataset_query_functions_v3.Ver3DatasetQueryFunctions(event_v3_storage_provider_utah)

### Columns

In [7]:
rfecv_selector_on_extra_trees__column_names = []

columns_list_file_pathname = os.path.join(model_data_snippets_dir, 'rfecv_selector_on_extra_trees__column_names.txt')
print(columns_list_file_pathname)
with open(columns_list_file_pathname, 'r') as columns_list_file:
    rfecv_selector_on_extra_trees__column_names = columns_list_file.read().splitlines()

ver4_machine_learning_w_labeled_flight_201906122/rfecv_selector_on_extra_trees__column_names.txt


In [8]:
rfecv_selector_on_extra_trees__column_names__special = []

special_columns_list_file_pathname = os.path.join(model_data_snippets_dir, 'rfecv_selector_on_extra_trees__column_names__special.txt')
print(special_columns_list_file_pathname)
with open(special_columns_list_file_pathname, 'r') as special_columns_list_file:
    rfecv_selector_on_extra_trees__column_names__special = special_columns_list_file.read().splitlines()

ver4_machine_learning_w_labeled_flight_201906122/rfecv_selector_on_extra_trees__column_names__special.txt


In [9]:
# This should be empty for now
rfecv_selector_on_extra_trees__column_names__special

[]

In [10]:
utah_columns_for_analysis_dict = query_functions_utah.get_columns_for_classification_dict__by_excluding(
    excluded_columns_re_list=('^.+$',),
    default_excluded_columns_re_list=[],
    included_columns_re_list=[('^$','source_file_(acquisition|trigger)(_full)?|global_gtu|packet_id|gtu_in_packet|event_id|num_gtu'),] + rfecv_selector_on_extra_trees__column_names
)

classification_utah_columns_for_analysis_dict = query_functions_utah.get_columns_for_classification_dict__by_excluding(
    excluded_columns_re_list=('^.+$',),
    default_excluded_columns_re_list=[],
    included_columns_re_list=rfecv_selector_on_extra_trees__column_names
)

print_columns_dict(utah_columns_for_analysis_dict)

miso2_2_spb_processing_v4.event
	- event_id
	- source_file_acquisition_full
	- source_file_trigger_full
	- source_file_acquisition
	- source_file_trigger
	- global_gtu
	- packet_id
	- gtu_in_packet
	- num_gtu

miso2_2_spb_processing_v4.event_trg_box_per_gtu
	- max
	- mean
	- mean_gz

miso2_2_spb_processing_v4.event_trg_pmt_per_gtu
	- max
	- mean
	- mean_gz

miso2_2_spb_processing_v4.event_trg_ec_per_gtu
	- mean
	- mean_gz

miso2_2_spb_processing_v4.event_trg_x_y_clusters
	- sizes_max
	- max_size_clu_width
	- max_size_clu_height
	- max_peak_clu_width
	- max_peak_clu_height
	- max_peak_clu_size

miso2_2_spb_processing_v4.event_trg_gtu_y_clusters
	- max_peak_clu_height

miso2_2_spb_processing_v4.event_orig_x_y
	- sum
	- max
	- mean
	- mean_gz

miso2_2_spb_processing_v4.event_proc1_x_y_clusters
	- count
	- sizes_min
	- sizes_max
	- clu_areas_max
	- max_size_clu_width
	- max_peak_clu_width
	- max_peak_clu_size

miso2_2_spb_processing_v4.event_proc2_x_y_clusters
	- sizes_min
	- sizes_max
	- 

WARNING: not selecting NULL trg lines

### Constructing the query

In [11]:
current_columns_for_analysis_dict = utah_columns_for_analysis_dict

utah_select_clause_str, utah_tables_list = \
    query_functions_utah.get_query_clauses__select({
    **current_columns_for_analysis_dict,
})

utah_clauses_str = \
    query_functions_utah.get_query_clauses__join(utah_tables_list)

utah_source_data_type_num = 2

utah_where_clauses_str = ''
# ''' 
#     AND abs(gtu_in_packet-42) < 20
#     AND {database_schema_name}.event_orig_x_y.count_nonzero > 256*6
# '''

for table, cols_list in classification_utah_columns_for_analysis_dict.items():
    for col in cols_list:
        utah_where_clauses_str += ' AND {}.{} IS NOT NULL\n'.format(table, col)

utah_events_selection_query = query_functions_utah.get_events_selection_query_plain(
    source_data_type_num=utah_source_data_type_num,
    select_additional=utah_select_clause_str, 
    join_additional=utah_clauses_str,
    where_additional=utah_where_clauses_str,
    order_by='{data_table_name}.event_id', 
    offset=0, limit=800000,
    base_select='')

In [12]:
# print(utah_events_selection_query)

In [13]:
utah_df = psql.read_sql(utah_events_selection_query, event_v3_storage_provider_utah.connection)

In [14]:
len(utah_df)

373622

In [ ]:
utah_df.head()

In [ ]:
utah_df[rfecv_selector_on_extra_trees__column_names].head()

In [ ]:
np.count_nonzero(utah_df['event_id'].isnull())

In [ ]:
utah_df['had_nan_fields'] = utah_df[rfecv_selector_on_extra_trees__column_names].isnull().any(axis=1)

In [ ]:
np.count_nonzero(utah_df['had_nan_fields'])

In [ ]:
utah_df_nonan = utah_df[~utah_df['had_nan_fields']]

In [ ]:
len(utah_df_nonan)

In [ ]:
# TODO
# SELECT COUNT(*) FROM spb_processing_v4_flatmap.event JOIN spb_processing_v4_flatmap.event_orig_x_y USING(event_id) WHERE source_data_type_num = 1 AND abs(gtu_in_packet-42) < 20 AND spb_processing_v4_flatmap.event_orig_x_y.count_nonzero > 256*6 LIMIT 5;
# SELECT COUNT( DISTINCT (source_file_acquisition, packet_id)) FROM spb_processing_v4_flatmap.event JOIN spb_processing_v4_flatmap.event_orig_x_y USING(event_id) WHERE source_data_type_num = 1 AND abs(gtu_in_packet-42) < 20 AND spb_processing_v4_flatmap.event_orig_x_y.count_nonzero > 256*6 LIMIT 5;


# Applying models

This model does not use scaled data

In [ ]:
# standard_scaler_on_train_rfecv_columns_pathname = \
#      os.path.join(model_data_snippets_dir, 'standard_scaler_on_train_rfecv_columns.pkl')
# standard_scaler_on_train_rfecv_columns = joblib.load(standard_scaler_on_train_rfecv_columns_pathname)

In [ ]:
# flight__rfecv_columns_scaled_X = \
#     standard_scaler_on_train_rfecv_columns.transform(
#         flight_df[rfecv_selector_on_extra_trees__column_names].values)
# if np.count_nonzero(flight_df['had_nan_fields']) > 0:
#     flight_nonan__rfecv_columns_scaled_X = \
#         standard_scaler_on_train_rfecv_columns.transform(
#             flight_df_nonan[rfecv_selector_on_extra_trees__column_names].values)
# else:
#     flight_nonan__rfecv_columns_scaled_X = flight__rfecv_columns_scaled_X

## Extra trees classifier

In [ ]:
utah_rfecv_columns__X  = utah_df_nonan[rfecv_selector_on_extra_trees__column_names].values

In [ ]:
extra_trees_cls_on_train_rfecv__model_plk_pathname = \
    os.path.join(model_data_snippets_dir, 'extra_trees_cls_on_train_rfecv.pkl')
extra_trees_cls_on_train_rfecv_est = joblib.load(extra_trees_cls_on_train_rfecv__model_plk_pathname)

In [ ]:
cls_column_base = 'extra_trees_cls_on_train_rfecv_est'
cls_column = cls_column_base + '_dropna'
cls_proba_column = 'extra_trees_cls_on_train_rfecv_est_dropna_proba'

In [ ]:
# this might not be correct (but for this particular selection it should be fine becaus utah_df_nonan == utah_df )
utah_df[cls_column_base] = \
    extra_trees_cls_on_train_rfecv_est.predict(utah_rfecv_columns__X)

utah_df[cls_column] = \
    ((utah_df[cls_column_base]==1) & ~utah_df['had_nan_fields']).astype('int8')

In [ ]:
utah_df[cls_proba_column] = np.nan
utah_df.loc[utah_df['event_id'].isin(utah_df_nonan['event_id']), cls_proba_column] = \
    extra_trees_cls_on_train_rfecv_est.predict_proba(utah_rfecv_columns__X)[:,1]

### Updating `flight_df_nonan` with classification predictions
(not in the report)

In [ ]:
utah_df_nonan = utah_df[~utah_df['had_nan_fields']]

### Statistics of selected events

In [ ]:
np.count_nonzero(utah_df[cls_column])

In [ ]:
np.count_nonzero(utah_df[cls_column])/len(utah_df)

In [ ]:
utah_df[[cls_column, cls_proba_column]].describe()

In [ ]:
for proba in np.arange(0.5, 1.0, 0.1):
    print('p > {:.2f}: {}'.format(proba, np.count_nonzero(utah_df[cls_proba_column] > proba)))

In [ ]:
utah_df_nonan[rfecv_selector_on_extra_trees__column_names].head()

### Saving utah data into tsv
(not in the report)

In [ ]:
save_utah_data_dump_file = True
overwrite_utah_data_dump_file = False

In [ ]:
if save_utah_data_dump_file:
    utah_data_tsv_pathname = os.path.join(data_snippets_dir, 'utah_data.tsv.gz')

    if overwrite_utah_data_dump_file or not os.path.exists(utah_data_tsv_pathname):
        print('Saving', utah_data_tsv_pathname)
        utah_df.to_csv(utah_data_tsv_pathname, sep='\t', compression='gzip')
    else:
        print('Already exists', utah_data_tsv_pathname)

# Visualization of the events

In [ ]:
shower_pred_by_proba_desc = utah_df_nonan[utah_df_nonan[cls_column] == 1].sort_values(cls_proba_column, ascending=False)
shower_pred_by_proba_asc = utah_df_nonan[utah_df_nonan[cls_column] == 1].sort_values(cls_proba_column, ascending=True)
noise_pred_by_proba_desc = utah_df_nonan[utah_df_nonan[cls_column] == 0].sort_values(cls_proba_column, ascending=False)
noise_pred_by_proba_asc = utah_df_nonan[utah_df_nonan[cls_column] == 0].sort_values(cls_proba_column, ascending=True)

### Visualization of tracks - air shower prediction - sorted by probability descending

In [ ]:
vis_events_df(
    shower_pred_by_proba_desc, 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

In [ ]:
vis_events_df(
    shower_pred_by_proba_desc[1000:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

In [ ]:
vis_events_df(
    shower_pred_by_proba_desc[5000:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

In [ ]:
vis_events_df(
    shower_pred_by_proba_desc[10000:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

In [ ]:
vis_events_df(
    shower_pred_by_proba_desc[20000:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

In [ ]:
vis_events_df(
    shower_pred_by_proba_desc[30000:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

In [ ]:
vis_events_df(
    shower_pred_by_proba_desc[40000:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

### Visualization of tracks - air shower prediction - sorted by probability ascending

In [ ]:
len(shower_pred_by_proba_asc)

In [ ]:
vis_events_df(
    shower_pred_by_proba_asc[:20], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

In [ ]:
vis_events_df(
    shower_pred_by_proba_asc[4000:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

In [ ]:
vis_events_df(
    shower_pred_by_proba_asc[8000:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

### Visualization of tracks - noise prediction - sorted by probability descending

In [ ]:
len(noise_pred_by_proba_desc)

In [ ]:
vis_events_df(
    noise_pred_by_proba_desc, 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

Shower events (20190409):
 - 1929 (short track, looks as single gtu track)
 
 661289

In [ ]:
vis_events_df(
    noise_pred_by_proba_desc[100:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition_full'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

Shower events (20190409):
 - 686894 (obvious air shower track)
 - 316757 (short obvious air shower track)

In [ ]:
vis_events_df(
    noise_pred_by_proba_desc[200000:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition_full'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

In [ ]:
vis_events_df(
    noise_pred_by_proba_desc[100000:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition_full'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

## Numbers of selected packets from eusospb analisi table

In [ ]:
eusospb_analisi_with_pathnames_left_pathname = os.path.join(utah_file_analysis_snippets_dir, 'eusospb_analisi_with_pathnames_left.tsv')
eusospb_analisi_with_pathnames_left_df = pd.read_csv(eusospb_analisi_with_pathnames_left_pathname, sep='\t', index_col=0)

In [ ]:
eusospb_analisi_with_pathnames_left_df

In [ ]:
df = eusospb_analisi_with_pathnames_left_df

for cfg in ['ta_euso', 'ta_euso_10', 'euso_bal', 'euso_bal_10', 'euso_bal_20']:
    df['max_' + cfg] = np.max([df[cfg + '_ec2'], df[cfg + '_ec5'], df[cfg + '_ec8']], axis=0)
    df['eff_' + cfg] = df['max_' + cfg] / df['pack']

In [ ]:
eusospb_analisi_with_pathnames_left_df

### Example on the first event

In [ ]:
shower_events = utah_df_nonan[(utah_df_nonan['source_file_acquisition'] == eusospb_analisi_with_pathnames_left_df.iloc[0]['file_pathname']) & (utah_df_nonan[cls_column] == 1)]

shower_events

In [ ]:
len(shower_events)

In [ ]:
shower_packets_num_events = shower_events.groupby('packet_id')['event_id'].count()

shower_packets_num_events

In [ ]:
len(shower_packets_num_events)

In [ ]:
multiple_times_packet_id_events_df = \
    shower_events[shower_events['packet_id'].isin(shower_packets_num_events[shower_packets_num_events > 1].index)].sort_values(['packet_id', cls_proba_column], ascending=[True,False])

multiple_times_packet_id_events_df

In [ ]:
vis_events_df(
    multiple_times_packet_id_events_df, 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition_full'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

### Counts of recognized showers

In [ ]:
# investigation of track gtu range - reuslt: 30 < gtu_in_packet < 68
#3793 5 32
# vis_events_df(
# #     utah_df_nonan.query('gtu_in_packet == 68').sort_values(cls_proba_column, ascending=False), 
#     utah_df_nonan.query('event_id == 364599'),
#     events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
#     close_after_vis=False, show=True, 
#     additional_printed_columns=[cls_proba_column, 'source_file_acquisition_full'],
#     by_one=True,
#     extension_func=None,
#     single_proj_width=4, single_proj_height=3
# )

In [ ]:
df = eusospb_analisi_with_pathnames_left_df

for class_num, class_label in enumerate(['noise', 'shower']):

    df['num_' + class_label + '_events'] = -1
    df['num_' + class_label + '_events_gtu_40'] = -1
    df['num_' + class_label + '_packets'] = -1
    df['num_' + class_label + '_packets_gtu_40'] = -1
    df['num_' + class_label + '_multiple_event_packets'] = -1
    df['num_' + class_label + '_multiple_packet_events'] = -1
    df['num_' + class_label + '_multiple_packet_events_gtu_40'] = -1

    for k, file_pathname in eusospb_analisi_with_pathnames_left_df['file_pathname'].items():

        cls_events = utah_df_nonan[(utah_df_nonan['source_file_acquisition'] == file_pathname) & (utah_df_nonan[cls_column] == class_num)]
        
        cls_packets_num_events = cls_events.groupby('packet_id')['event_id'].count()
        num_packets = len(cls_packets_num_events)
        
        cls_events_gtu_40_df = \
            cls_events[
                (30 < cls_events['gtu_in_packet']) & \
                (cls_events['gtu_in_packet'] < 68)
            ]
        
        cls_packets_num_events_gtu_40 = cls_events_gtu_40_df.groupby('packet_id')['event_id'].count()
        num_packets_gtu_40 = len(cls_packets_num_events_gtu_40)
        
        multple_event_packets_gtu_40_indexes = cls_packets_num_events_gtu_40[cls_packets_num_events_gtu_40 > 1].index
        
        if(len(multple_event_packets_gtu_40_indexes) > 1):
            print('!!! MULTIPLE EVENTS IN A SINGLE PACKET\n\t{}({}) / {}({}): {}'.format(
                class_label, class_num, file_pathname, k, len(multple_event_packets_gtu_40_indexes),
                # 'cls_packets_num_events_gtu_40[cls_packets_num_events_gtu_40 > 1]'
            ))
        
        multple_event_packets_indexes = cls_packets_num_events[cls_packets_num_events > 1].index

        multiple_packet_events_df = \
            cls_events[cls_events['packet_id'].isin(multple_event_packets_indexes)] \
                .sort_values(['packet_id', cls_proba_column], ascending=[True,False])

        multiple_packet_events_gtu_40_df = \
            multiple_packet_events_df[
                (36 < multiple_packet_events_df['gtu_in_packet']) & \
                (multiple_packet_events_df['gtu_in_packet'] < 50)
            ]

        
        if num_packets_gtu_40 < num_packets and class_num == 1:
                        
            cls_events_outside_gtu_40_df = \
                cls_events[
                    (30 >= cls_events['gtu_in_packet']) | \
                    (cls_events['gtu_in_packet'] >= 68)
                ]
            
            print('!!! (only shower) MORE PACKETS OUTSIDE GTU=40\n\t{}({}) / {}({}): num_packets_gtu_40={}, num_packets={}'.format(
                class_label, class_num, file_pathname, k, num_packets_gtu_40, num_packets
            ))
            for ti, (tk, tr) in enumerate(cls_events_outside_gtu_40_df.iterrows()):
                print('\t', tr['event_id'], tr['packet_id'], tr['gtu_in_packet'])
                if ti > 5:
                    break
        
        # multiple_packet_events_proba_09_df = multiple_packet_events_df[multiple_packet_events_df[cls_proba_column] > 0.9] 

        df.loc[k, 'num_' + class_label + '_events'] = len(cls_events)
        df.loc[k, 'num_' + class_label + '_events_gtu_40'] = len(cls_events_gtu_40_df)
        df.loc[k, 'num_' + class_label + '_packets'] = num_packets
        df.loc[k, 'num_' + class_label + '_packets_gtu_40'] = num_packets_gtu_40
        df.loc[k, 'num_' + class_label + '_multiple_event_packets'] = len(multple_event_packets_indexes)
        df.loc[k, 'num_' + class_label + '_multiple_packet_events'] = len(multiple_packet_events_df)
        df.loc[k, 'num_' + class_label + '_multiple_packet_events_gtu_40'] = len(multiple_packet_events_gtu_40_df)

In [ ]:
df = eusospb_analisi_with_pathnames_left_df

for col in ['shower_packets', 'shower_packets_gtu_40']:
    df['eff_' + col] = df['num_' + col] / df['pack']

In [ ]:
eusospb_analisi_with_pathnames_left_df[[
    'num_lens', 'run', 'per', 'mj', 'pack', 
    # 'euso_bal_ec2', 'euso_bal_ec5', 'euso_bal_ec8', 
    'max_euso_bal', 'max_euso_bal_10', 
    'num_noise_packets', 'num_shower_packets', 
    'num_noise_packets_gtu_40', 'num_shower_packets_gtu_40',
    'eff_euso_bal_10', 'eff_shower_packets_gtu_40',
    'num_noise_events', 'num_shower_events',
    'num_noise_events_gtu_40', 'num_shower_events_gtu_40', 
    'acq_group', 'file_pathname'
]].sort_values('mj', ascending=True)

In [ ]:
eusospb_analisi_with_pathnames_left_df.query('num_noise_packets == 0 and num_shower_packets == 0')

In [ ]:
utah_df_nonan.query('source_file_acquisition == "300916/GLS/allpackets-SPBEUSO-ACQUISITION-20161004-033506-001.001--45degaway55per.root"')

In [ ]:
print('Number of uncassified / missing files:', len(eusospb_analisi_with_pathnames_left_df.query('num_noise_packets == 0 and num_shower_packets == 0')))

In [ ]:
def acq_group_stats(num_lens, acq_group=None):
    
    print('Num lens:', num_lens)
    print('Acquisitions group:', acq_group)
    
    eusospb_analisi_2_len_by_mj = \
        eusospb_analisi_with_pathnames_left_df \
            .query('num_lens == {num_lens} and (num_noise_events > 0 or num_shower_events > 0) {acq_group_part}' \
                       .format(num_lens=num_lens, acq_group_part='and acq_group=="{}"'.format(acq_group) if acq_group else '')
                  ) \
            .sort_values('mj', ascending=True)
    
    print('Number of entries:', len(eusospb_analisi_2_len_by_mj))
    
    IPython.display.display(eusospb_analisi_2_len_by_mj[[
        'num_lens', 'run', 'per', 'mj', 'pack', 
        'max_euso_bal', 'max_euso_bal_10', 
        'num_noise_packets', 'num_shower_packets', 
        'num_noise_packets_gtu_40', 'num_shower_packets_gtu_40',
        'eff_euso_bal_10', 'eff_shower_packets_gtu_40',
        'acq_group', 'file_pathname'
    ]])
    
    fig, ax = plt.subplots(figsize=(8,6))
    eusospb_analisi_2_len_by_mj.plot('mj', 'eff_euso_bal', ax=ax, label='EUSO Bal. (P=1,R=1)')
    eusospb_analisi_2_len_by_mj.plot('mj', 'eff_euso_bal_10', ax=ax, label='EUSO Bal. (P=1,R=1) -10%')
    eusospb_analisi_2_len_by_mj.plot('mj', 'eff_shower_packets_gtu_40', ax=ax, label='Classification model ({})'.format(classification_id))
    ax.set_xlabel('Laser energy [mJ]')
    ax.set_ylabel('Efficiency')
    
    path_compatible_acq_group = slugify(acq_group) if acq_group else 'any'
    
    fig.savefig(os.path.join(data_snippets_dir, 
                             'efficiency_plot_acq_group_{}_eusobal_classification_model_comparison.svg' \
                                 .format(path_compatible_acq_group)))
    
    fig, ax, errbr = \
        plot_efficiency_stat_simple(
            eusospb_analisi_2_len_by_mj['mj'], eusospb_analisi_2_len_by_mj['eff_shower_packets_gtu_40'], 
            yerr=statmodels_proportion_confint(eusospb_analisi_2_len_by_mj['num_shower_packets_gtu_40'].data, 
                                               eusospb_analisi_2_len_by_mj['pack'].data, method='beta'),
            figsize=(8,6), ylabel='Efficiency', xlabel='Laser energy [mJ]',
            show=False
        );
    
    fig.savefig(os.path.join(data_snippets_dir, 
                             'efficiency_plot_acq_group_{}_classification_model.svg' \
                                 .format(path_compatible_acq_group)))
    
    plt.show()

In [ ]:
for num_lens in np.unique(eusospb_analisi_with_pathnames_left_df['num_lens']):
    for acq_group in (None, *eusospb_analisi_with_pathnames_left_df.query('num_lens=='+str(num_lens))['acq_group'].unique().tolist()):
        acq_group_stats(num_lens, acq_group)
        print()
        print('='*100)
        print()

In [ ]:
def failed_mj(num_lens, acq_group=None, find_events=True, show_events=False, events_per_figure=10):
    
    print('Num lens:', num_lens)
    print('Acquisitions group:', acq_group)
    
    eusospb_analisi_2_len_by_mj = \
        eusospb_analisi_with_pathnames_left_df \
            .query('num_lens == {num_lens} and (num_noise_events > 0 or num_shower_events > 0) {acq_group_part} and eff_shower_packets_gtu_40 < 0.5 ' \
                       .format(num_lens=num_lens, acq_group_part='and acq_group=="{}"'.format(acq_group) if acq_group else '')
                  ) \
            .sort_values('mj', ascending=True)
    
    print('Number of entries:', len(eusospb_analisi_2_len_by_mj))
    
    IPython.display.display(eusospb_analisi_2_len_by_mj[[
        'num_lens', 'run', 'per', 'mj', 'pack', 
        'max_euso_bal', 'max_euso_bal_10', 
        'num_noise_packets', 'num_shower_packets', 
        'num_noise_packets_gtu_40', 'num_shower_packets_gtu_40',
        'eff_euso_bal_10', 'eff_shower_packets_gtu_40',
        'acq_group', 'file_pathname'
    ]])
    
    if not find_events:
        return

    for i, r in eusospb_analisi_2_len_by_mj.iterrows():
        file_pathname = r['file_pathname']
        print(file_pathname)
        print('\teff_euso_bal = {:.3f}\n\teff_euso_bal_10 = {:.3f}\n\teff_shower_packets_gtu_40 = {:.3f}\n\tmj = {:.3f}\n\tpack = {:d}'.format(
            r['eff_euso_bal'], r['eff_euso_bal_10'], r['eff_shower_packets_gtu_40'], r['mj'], r['pack']
        ))
#         all_file_entries = eusospb_analisi_with_pathnames_left_df.query('source_file_acquisition == "{}"'.format(file_pathname))
#         print('Number of all entries:', len(all_file_entries))
        
        acq_events = utah_df_nonan[(utah_df_nonan['source_file_acquisition'] == file_pathname)]
        
        for class_num, class_label in enumerate(['noise', 'shower']):
            
            cls_events = acq_events[(acq_events[cls_column] == class_num)]
            cls_packets_num_events = cls_events.groupby('packet_id')['event_id'].count()
            
            cls_events_gtu_40_df = \
                cls_events[
                    (30 < cls_events['gtu_in_packet']) & \
                    (cls_events['gtu_in_packet'] < 68)
                ].sort_values(cls_proba_column, ascending=bool(class_num))
            
            cls_packets_num_events_gtu_40 = cls_events_gtu_40_df.groupby('packet_id')['event_id'].count()
            
            print()
            print(
                'Events classified as {}:\n\tcount:        {:5d} (30 < gtu_in_packet < 68)\n\tpacket count: {:5d} (30 < gtu_in_packet < 68)' \
                    .format(class_label, len(cls_events_gtu_40_df), len(cls_packets_num_events_gtu_40))
            )
            print()
            
            if show_events:
                vis_events_df(
                    cls_events_gtu_40_df, 
                    events_per_figure=events_per_figure, max_figures=1, vis_gtux=True, vis_gtuy=True, 
                    close_after_vis=False, show=True, 
                    additional_printed_columns=[cls_proba_column, 'source_file_acquisition_full'],
                    by_one=True,
                    extension_func=None,
                    single_proj_width=4, single_proj_height=3
                )
            print('-'*100)
    

In [ ]:
for num_lens in np.unique(eusospb_analisi_with_pathnames_left_df['num_lens']):
    for acq_group in eusospb_analisi_with_pathnames_left_df.query('num_lens=='+str(num_lens))['acq_group'].unique().tolist():
        failed_mj(num_lens, acq_group, find_events=True, show_events=True)
        print()
        print('='*100)
        print()
        

In [ ]:
#event_v3_storage_provider_flight.connection.reset()

In [ ]:
# flight_nonan_classified_shower_pathname = os.path.join(data_snippets_dir, 'flight_nonan_classified shower.tsv')

In [ ]:
# flight_df_nonan[flight_df_nonan['extra_trees_cls_on_train_kbest400_128_est_dropna']==1].to_csv(flight_nonan_classified_shower_pathname, sep='\t')

In [ ]:
# TODO select clusters with positive classification (sort by the number of classifications) show distribution of event types

In [ ]:

# flight_df_nonan_subset[['dbscan_tsne_y','manual_classification_class_number'].hist('dbscan_tsne_y', figsize=(24,4), bins=2*len(dbscan_on_tsne_classes)+1)

# plt.show()

In [ ]:
# THIS IS NOT WHAT IS DESIRED - values should be split into features ?
# flight_nonan__cls_tsneclu_corr_df = \
#     flight_df_nonan[['dbscan_tsne_y', 'manual_classification_class_number']].corr()
# f, ax = plt.subplots(figsize=(28,22))
# plt.close('all')
# sns.heatmap(flight_nonan__cls_tsneclu_corr_df, cmap='inferno', annot=True)
# plt.show()

In [ ]:
#     f, ax = plt.subplots()
#     f.set_size_inches(8,4)
#     flight_df_nonan_subset[['dbscan_tsne_y', 'manual_classification_class_number']].plot.bar(by='dbscan_tsne_y', ax=ax)
    

In [ ]:
# flight_nonan__tsne__gmm_y_pred = gmm.predict(flight_df_nonan[['tsne_X_0','tsne_X_1']].values)

In [ ]:
# flight_data__k50best_var_th_scaled_X = \
#     k50best_f_classif_selector_on_var_th_sc_train.transform(
#         var_th_selector_on_scaled_train.transform(
#             standard_scaler_on_train.transform(
#                 unl_flight_df[analyzed_common_df_columns].dropna().values)
#         )
#     )

# extra_trees_classifier_on_train_kbest50__X_flight = flight_data__k50best_var_th_scaled_X
# extra_trees_classifier_on_train_kbest50__y_flight_pred = \
#     extra_trees_classifier_on_train_kbest50.predict(extra_trees_classifier_on_train_kbest50__X_flight)